Импорт библиотек и загрузка датасета

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier

In [2]:
data = pd.read_csv(r'../data/car-evaluation_preprocessed.csv', sep=',')

In [3]:
data

,buying,maint,doors,persons,lug_boot,safety,class
0,4,4,2,2,1,1,1
1,4,4,2,2,1,2,1
2,4,4,2,2,1,3,1
3,4,4,2,2,2,1,1
4,4,4,2,2,2,2,1
...,...,...,...,...,...,...,...
1723,1,1,5,5,2,2,3
1724,1,1,5,5,2,3,4
1725,1,1,5,5,3,1,1
1726,1,1,5,5,3,2,3


In [4]:
X = data.iloc[:, :-1].values
y = data.iloc[:, 6].values

In [5]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [6]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [7]:
def print_classification_model_metrics(estimator, y_test, y_pred):
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    print(estimator.score(X_test, y_test))

# DTC

In [8]:
%%time
parameters_dtc = {'max_depth': np.arange(1,20,1)}
dtc = DecisionTreeClassifier()
dtc_base = GridSearchCV(dtc, parameters_dtc).fit(X_train, y_train)
dtc_base.best_params_

Wall time: 231 ms


{'max_depth': 11}

In [9]:
print_classification_model_metrics(dtc_base, y_test, dtc_base.predict(X_test))

[[259   1   0   0]
 [  0  57   3   1]
 [  0   1  10   0]
 [  0   0   0  14]]
              precision    recall  f1-score   support

           1       1.00      1.00      1.00       260
           2       0.97      0.93      0.95        61
           3       0.77      0.91      0.83        11
           4       0.93      1.00      0.97        14

    accuracy                           0.98       346
   macro avg       0.92      0.96      0.94       346
weighted avg       0.98      0.98      0.98       346

0.9826589595375722


# Бэггинг

In [10]:
%%time

UsageError: %%time is a cell magic, but the cell body is empty. Did you mean the line magic %time (single %)?
